# Infiltrationsbestimmung nach Green und Ampt

Mit Green-Ampt-Modell lässt sich die kumulative Infiltration $ I(t)$ in den Boden als Funktion der Zeit $t$ beschreiben:.

$$ I(t) - \Delta\theta \Delta h \ln \left( 1 + \frac{I(t)}{\Delta h} \right) = K_s t $$

$$t = \frac{I(t) - \Delta\theta \Delta h \ln \left(1 + \frac{I(t)}{\Delta\theta \Delta h} \right)}{K_s}$$

wobei: $t$...Zeit [min], $i(t)$...kummulative Infiltration, $\Delta\theta$...Differenz zwischen dem initialen und durchfeuchteten Bodenwassergehalt, $\Delta h$...Differenz zwischen dem initialen und finalen Matrixpotenzial, sowie $K_s$...gesättigte hydraulische Leitfähigkeit des Bodens.

### Werte für verschiedene Bodenparameter

| Korngrößen | Gesättigte hydraulische <br> Leitfähigkeit Ks [mm/h] | Saugspannung an der <br> Feuchtefront [mm] | Porosität [-] | Feldkapazität [mm] | Welkepunkt [mm] |
|------------|:--------------------------------------:|:-----------------:|:-------------:|:-------------:|:----------:|
| Sand | 120 | 49-150 | 0,437 | 0,062 | 0,024 |
| lehmiger Sand | 30 | 61-250 | 0,437 | 0,105 | 0,047 |
| sandiger Lehm | 11 | 110-250 | 0,453 | 0,190 | 0,085 |
| Lehm | 3 | 89-350 | 0,463 | 0,232 | 0,116 |
| Schluffiger Lehm | 7 | 170 | 0,501 | 0,284 | 0,135 |
| sandig-toniger Lehm | 2 | 220 | 0,398 | 0,244 | 0,136 |
| toniger Lehm | 1 | 210 | 0,464 | 0,310 | 0,187 |
| schluffig-toniger Lehm | 1 | 270 | 0,471 | 0,342 | 0,210 |
| sandiger Ton | 1 | 240 | 0,430 | 0,321 | 0,221 |
| schluffiger Ton | 1 | 290 | 0,479 | 0,371 | 0,251 |
| Ton | 0,3 | 320-1000 | 0,475 | 0,378 | 0,265 |

nach: D.A. Chin: Water-Resources Enginieering (2013)


In [1]:
import numpy as np
import matplotlib.pyplot as plt
import ipywidgets as widgets
from ipywidgets import interact,interactive, fixed, interact_manual, Button, Layout
from IPython.display import clear_output


In [2]:


class GreenAmptModel:                                                                   # Klasse mit Methoden zur Berechnung und Visualisierung der Infiltration nach Green-Ampt (1911)

    def __init__(self):             
        self.reference_lines = []                                                                       # Initialisierung von Parametern bzw. Listen, in denen später berechnete
        self.ref_line_params = []                                                                       # Werte gespeichert werden können.
        self.short_time_ref = []                                                                        # Durch die Initialisierung sind die mit "self" markierten Variablen für alle
        self.short_time_params = []                                                                     # Methoden in der Klasse: GreenAmptModel sichtbar.
        self.infiltration_rate_ref = []
        self.infiltration_ref = []


    def t_for_It(self, I_t, d_theta, d_h, Ks=0.3):                                                      # Methode zur Berechnung der Zeitschritte zwischen einzelnen Infiltrationsschrittenb(t)
            return (I_t - d_theta * d_h * np.log(1+ (I_t/(d_theta*d_h))))/Ks                                    # Ausgabe: np.array

    def long_time_approximation(self, K, x):                                                            # Methode zur Berechnung der Langzeitannäherung (i_t_long) an die Infiltrationsrate (i_t)
        it_long = np.array([K]*len(x))                                                                          # Ausgabe: np.array
        return it_long

    def d_h(self, h0=0, hf=600):                                                                        # Methoden zur Berechnung einer Differenz zwischen zwei Werten
        return hf-h0                                                                                            # Ausgabe: Float

    def d_theta(self, theta_0=0.44, theta_i=0.1):
        return theta_0 - theta_i
    
    def infiltration_rate(self, new_t, t, I_t):                                                         # Methode zur Berechnung der Infiltationsate 
        y2 = np.interp(new_t, t, I_t)                                                                           # Ausgabe: np.array
        i_t = np.diff(y2)
        #self.i_t = i_t
        return i_t
    
    def read_datastring(self, y_data_string='', x_data_string=''):                                      # Methode zum Einlesen von Messdaten
        if y_data_string and x_data_string != '':                                                                # Die Messdaten werden als String eingelesen und in Listen umgewandelt
            y_list = y_data_string.split(',')                                                                   # Ausgabe: 2 Listen mit Funktionswerten
            x_list = x_data_string.split(',')
            x_data = [eval(i) for i in x_list]
            y_data = [eval(i) for i in y_list]  
            return y_data, x_data


    def plot_scenario(self, I_data_string='', tI_data_string='', i_data_string='', ti_data_string='',   # Methode zum Plotten der Daten
                       h0_hf = (-60,0), thetai_theta0 = (0.2,0.44), Ks=4):

        h0 = h0_hf[0]                                                                                   # Vom Benutzer eingelesene Werte werden in Variablen gespeichert
        hf = h0_hf[1]
        theta_i = thetai_theta0[0]
        theta_0 = thetai_theta0[1]
        
                                                                                                        # Aus den definierten Methoden werden Daten generiert
        delta_theta = self.d_theta(theta_0, theta_i)                                                        # delta theta (type: Float)
        delta_h = self.d_h(h0, hf)                                                                          # delta h (type: Int)
        I_t = np.arange(0.1, 260, 0.1)                                                                      # Infiltrationsdaten (type: np.array)
        t = self.t_for_It(I_t, delta_theta, delta_h, Ks)                                                    # Zeitschritte für die Infiltration (type: np.array)
        self.new_t = np.arange(round(t[0], 1), round(t[-1], 1), 0.01)                                       # Zeitschritte für die Infiltration auf 1. Dezimalstelle gerundet (type: np.array)
        self.i_t = self.infiltration_rate(self.new_t, t, I_t)                                               # Infiltrationsrate (type: np.array)
        i_t_long = self.long_time_approximation(Ks, self.new_t)                                             # Langzeitannäherung der Infiltrationsrate ~ Ks (type: np.array)


        fig, (ax1, ax2) = plt.subplots(2, figsize=(12,12))                                              # Eine Abbildung für die Plots wird erstellt


        for ref, params in zip(self.infiltration_ref, self.ref_line_params):                            # In on_button_click() definierte Referenzlinien für I_t werden geplottet
            ref_x, ref_y = ref
            ax1.plot(ref_x, ref_y, linestyle='-.', alpha=0.6, 
                     label='I(t): hf:{}, h0:{}, theta_i:{}, theta_0:{}, Ks:{}'.format(params[0], params[1], params[2], params[3], params[4]), linewidth=2.5)
        
        for ref, params in zip(self.infiltration_rate_ref, self.ref_line_params):                       # In on_button_click() definierte Referenzlinien für I_t werden geplottet
            ref_x, ref_y = ref
            ax2.plot(ref_x, ref_y, linestyle='-.', alpha=0.6,
                      label='i(t): hf:{} h0:{} theta_i:{} theta_0:{} Ks:{}'.format(params[0], params[1], params[2], params[3], params[4]), linewidth=2.5)


        ax1.plot(t, I_t, color='black', label='I(t): hf:{}, h0:{}, theta_i:{}, theta_0:{}, Ks:{}'.format(hf, h0, theta_i, theta_i, theta_0, Ks), linewidth=1)                                      # Plotten der Infiltrationsdaten auf einer Achse 
        
        ax1.set_title(label='Green-Ampt Infiltrationsmodell', fontsize=20, loc= 'center')                       # Grenzen für die x- und y-Achse festlegen
        ax1.set_ylim(0.1, max(I_t))
        ax1.set_xlim(0, 185)
        ax1.set_xlabel('Zeit [min]', fontsize=10)                                                               # Achsenbeschreibung hinzufügen
        ax1.set_ylabel('Infiltration [mm/min]', fontsize=10)
        ax1.grid(which = "major", linewidth = 1)                                                                # Gitternetz hinzufügen
        ax1.grid(which = "minor", linewidth = 0.2)
        ax1.minorticks_on()
        ax1.legend(loc='best')                                                                                  # Legende einblenden


        ax2.plot(self.new_t[1:-5], self.i_t[0:-5]*100, color='black', label='i(t): hf:{}, h0:{}, theta_i:{}, theta_0:{}, Ks:{}'.format(hf, h0, theta_i, theta_i, theta_0, Ks), linewidth=1)        # Plotten der Infiltrationsrate auf einer weiteren Achse
        ax2.plot(self.new_t, i_t_long, 'red', linestyle='--',                                           # Plotten der Langzeitnäherung auf derselben Achse
                  label='Long time approximation ~ Ks [mm/min]')
  
        ax2.set_ylim(0.1, 150)                                                                                  # Grenzen für die x- und y-Achse festlegen
        ax2.set_xlim(0, max(self.new_t))
        ax2.set_yscale('log')                                                                                   # y-Achsenskalierung logarithmisch setzen
        ax2.set_xlabel('Zeit [min]', fontsize=10)                                                               # Achsenbeschreibung hinzufügen
        ax2.set_ylabel('Infiltrationsrate [mm/min]', fontsize=10)
        ax2.grid(which = "major", linewidth = 1)                                                                # Gitternetz hinzufügen
        ax2.grid(which = "minor", linewidth = 0.2)
        ax2.minorticks_on()
        ax2.legend(loc='best')                                                                                  # Legende einblenden

                                                                                                       # Plotten von zusätzlichen, vom Benutzer eingegebenen Messdaten
                                                                                                        ## für die Infiltration
        if I_data_string and tI_data_string != '':                                                               # Wurden Messdaten übergeben? Wenn, ja:
            I_data, tI_data = self.read_datastring(I_data_string, tI_data_string)                                # Umwandeln von str -> list                       
            ax1.scatter(tI_data, I_data, color='black', linestyle='', marker='x')                                 # Plotten

        if i_data_string and ti_data_string != '':                                                               # Wurden Messdaten übergeben? Wenn, ja:
            i_data, ti_data = self.read_datastring(i_data_string, ti_data_string)                                                            # Umwandeln von str -> list
            ax2.scatter(ti_data, i_data, color='black', linestyle='', marker='x')                                 # Plotten


        if self.on_sfbutton_click(self.sf_button) == True:                                              # Wenn der Button gedrückt ist, wird die Abbildung gespeichert
            fig.savefig('Green_Ampt_Infiltration.png')

        plt.show()

    def on_button_click(self, button):
  
        h0_val = self.h0_hf.value[0]                                                                    # Speichern der über "Widgets" eingelesenen Werte in Variablen
        hf_val = self.h0_hf.value[1]
        theta_i_val = self.thetai_theta0.value[0]
        theta_0_val = self.thetai_theta0.value[1]
        Ks_val = self.Ks_slider.value

        delta_theta = (self.d_theta(theta_0_val, theta_i_val))                                          # Berechnen der Werte mittels oben definierter Methoden
        delta_h = self.d_h(h0_val, hf_val)
        I_t = np.arange(0, 260, 0.1)
        t = self.t_for_It(I_t, delta_theta, delta_h, Ks_val)
        self.new_t = np.arange(round(t[0], 1), round(t[-1], 1), 0.01)
        i_t = self.infiltration_rate(self.new_t, t, I_t)

        
        self.reference_lines.append((t, I_t))                                                           # Speichern der gerade vom Benutzer eingestellten Funktionsverläufe als np.arrays
        self.ref_line_params.append((h0_val, hf_val, theta_0_val, theta_i_val, Ks_val))                 # ebenfalls für die Parameter
        self.reference_lines.append((self.new_t, i_t))
        self.ref_line_params.append((h0_val, hf_val, theta_0_val, theta_i_val, Ks_val))
        self.infiltration_ref.append((t, I_t))
        self.infiltration_rate_ref.append((self.new_t[1:-5], self.i_t[:-5]*100))

        clear_output(wait=True)                                                                         # Alten Plot löschen und neuen inklusive Referenzlinien Plotten
        display(self.interactive_plot)
        display(button)
        display(self.sf_button)

    def on_sfbutton_click(self, sf_button):
        return True                                                                                     # Übergibt einen Wahrheitswert, wenn der Button gedrückt ist


    def create_interactive_plot(self):
        
        button = Button(description="Referenzlinie Speichern",                                                      # Erstellen eines Buttons zum Speichern der Referenzlinien
                         layout=Layout(width='20%', height='40px'), button_style='info')
        button.on_click(self.on_button_click)                                                                               # Funktion für den Fall des 'gedrückten buttons'

        self.sf_button = Button(description="Abbildung Speichern",                                                  #Button zum Speichern der Abbildung
                                layout=Layout(width='20%', height='40px'), button_style='info')                                                  
        self.sf_button.on_click(self.on_sfbutton_click)

        self.h0_hf = widgets.IntRangeSlider(                                                            # Ein IntRangeSlider zum Einlesen von hf und h0 über den Benutzer(übergibt 2 Werte als tuple)
            value=[0, 600], min=0, max=1000, step=1,
            description=('h0 - hf [mm]'),style = {'description_width': 'initial'}
            )
        self.thetai_theta0 = widgets.FloatRangeSlider(                                                  # Ein FloatRangeSlider für theta i und theta 0
            value=[0.1, 0.4], min=0, max=0.5, step=0.01,
            description=('theta i - theta 0 [-]'), style = {'description_width': 'initial'}
            )
        self.Ks_slider = widgets.FloatSlider(                                                           # Ein FloatSlider für hf und h0 (übergibt einen Float)
            value=0.5, min=0.01, max=5, step=0.01,
            description='Ks [mm/min]' , style = {'description_width': 'initial'}
            )

    
        self.I_data_string = widgets.Textarea(                                                          # 4 Textfelder zum Einlesen von Messdaten (übergibt jeweils einen String)
            placeholder='Daten Bitte mit . als Dezimaltrennzeichen und , als Trennzeichen eingeben',
            description='Infiltration I [mm]', style = {'description_width': 'initial'}
            )
        
        self.tI_data_string = widgets.Textarea(
            placeholder='Daten Bitte mit . als Dezimaltrennzeichen und , als Trennzeichen eingeben',
              description='Zeitdaten für I [min]', style = {'description_width': 'initial'}
              )

        self.i_data_string = widgets.Textarea(
            placeholder='Daten Bitte mit . als Dezimaltrennzeichen und , als Trennzeichen eingeben',
              description='Infiltrationsate i [mm/min]', style = {'description_width': 'initial'}
              )
        
        self.ti_data_string = widgets.Textarea(
            placeholder='Daten Bitte mit . als Dezimaltrennzeichen und , als Trennzeichen eingeben',
              description='Zeitdaten für i [min]', style = {'description_width': 'initial'}
              )

        self.interactive_plot = interactive(                                                             # Verknüpfen der interaktiven Eingabewidgets mit dem Plot
            self.plot_scenario, 
            I_data_string=self.I_data_string, tI_data_string=self.tI_data_string, 
            i_data_string=self.i_data_string, ti_data_string=self.ti_data_string, 
            h0_hf = self.h0_hf, thetai_theta0 = self.thetai_theta0, Ks=self.Ks_slider
            )
        
        display(self.interactive_plot)                                                                   # Aufruf von Plot und Button        
        display(button)
        display(self.sf_button)



In [3]:
GreenAmptModel().create_interactive_plot()


interactive(children=(Textarea(value='', description='Infiltration I [mm]', placeholder='Daten Bitte mit . als…

Button(button_style='info', description='Referenzlinie Speichern', layout=Layout(height='40px', width='20%'), …

Button(button_style='info', description='Abbildung Speichern', layout=Layout(height='40px', width='20%'), styl…